The first part of the assignment is importing the packages

In [1]:
import (
	"encoding/csv"
    "gonum.org/v1/gonum/floats"
	"fmt"
	"log"
	"os"
	"sort"
	"strconv"
    "context"
	"io/ioutil"
	"log"
	"strings"
	"github.com/rocketlaunchr/dataframe-go"
	"github.com/rocketlaunchr/dataframe-go/imports"
)

The second part of the assignment is defining the main function in which we call the LoadData, parseRecords, and recommendBasedOnLocation functions. But of course, we need to run other functions.

In [15]:
func main() {
    // Load the data
    records := loadData()
    
    // Parse the records into locations
    locations := parseRecords(records)

    // Recommend locations based on a given location
    recommendations := recommendBasedOnLocation(locations, "Ideal Location")
	if recommendations != nil {
		fmt.Println("Top recommended locations:")
		for _, rec := range recommendations[:5] {
			fmt.Printf("%s (Similarity: %.2f)\n", rec.Location.Name, rec.Similarity)
		}
	}
}

Top recommended locations:
Stikker Creek (Similarity: 0.95)
Flowering Fields (Similarity: 0.86)
Peaceful Meadows (Similarity: 0.85)
Quiet Glades (Similarity: 0.61)
Serene Meadow (Similarity: 0.55)


The third part of the assignment is defining the load function, we need that to read and return the records.

In [9]:
// Open the CSV file and create a new CSV reader reading from the opened file
func loadData() [][]string {
    // Open the CSV file
    f, err := os.Open("gopher_locations.csv")
    if err != nil {
        log.Fatalf("Cannot open 'locations.csv': %s\n", err.Error())
    }
    defer f.Close()

    // Create a new CSV reader reading from the opened file
    reader := csv.NewReader(f)

    // Read all the records from the CSV file
    records, err := reader.ReadAll()
    if err != nil {
        log.Fatalf("Cannot read CSV data: %s\n", err.Error())
    }

    return records
}

The next part of the assignment is defining the structure of the data, we need that because we are going to use the data in the CSV file and by defining the structure we can easily access the data
1) Name: Name of the location
2) AverageTemperature: Average temperature of the location
3) NrPredators: Number of predators in the location
4) NrFoodSources: Number of food sources in the location
5) NrWaterSources: Number of water sources in the location
6) NrHumans: Number of humans in the location
7) VegetationType: Type of vegetation in the location

In [10]:
// Define a structure to hold location data
type Location struct {
	Name             string
	AverageTemperature      float64
	NrPredators  float64
	NrFoodSources float64
	NrWaterSources     float64
	NrHumans    float64
	VegetationType       float64
	LandSize         float64
}

The fifth part of the assignment is defining the function to encode the categorical variables, we need that to encode the VegetationType column in the CSV file because the VegetationType column contains categorical variables.

In [11]:
// Helper function to encode categorical variables for VegetationType column
func encodeCategory(value string, categories map[string]float64) float64 {
	return categories[value]
}

The sixth part of the assignment is defining the function to parse the records, we need that to parse the records in the CSV file into locations.

1) The function takes one parameter, the records
2) The function returns a list of locations
3) The function loops through the records and parses the records into locations
4) The function uses the encodeCategory function to encode the VegetationType column since it is the only categorical variable in the CSV file

In [12]:
// Parse the records into locations
func parseRecords(records [][]string) []Location {
    locations := []Location{}
    vegetationCategories := map[string]float64{"Fields": 0, "Meadow": 1, "Hills": 2, "Forest": 3, "Wetland": 4, "Savanna": 5, "Desert": 6}

    for _, record := range records[1:] { // Skip header
        averageTemp, _ := strconv.ParseFloat(record[1], 64)
        nrPredators, _ := strconv.ParseFloat(record[2], 64)
        nrFoodSources, _ := strconv.ParseFloat(record[3], 64)
        nrWaterSources, _ := strconv.ParseFloat(record[4],64)
        nrHumans, _ := strconv.ParseFloat(record[5],64)
        vegetationType := encodeCategory(record[6], vegetationCategories)
        landSize, _ := strconv.ParseFloat(record[7], 64)

        location := Location{
            Name:             record[0],
            AverageTemperature:      averageTemp,
            NrPredators:  NrPredators,
            NrFoodSources: nrFoodSources,
            NrWaterSources:     nrWaterSources,
            NrHumans:    nrHumans,
            VegetationType:       vegetationType,
            LandSize:         landSize,
        }
        locations = append(locations, location)
    }
    return locations
}

In the next part of the assignment we define the function to calculate the similarity score, we need that to calculate the similarity between the ideal location and the other locations.
1) The function takes two parameters, the ideal location and the other location
2) The function returns the similarity score
3) The function calculates the similarity score by normalizing the features and then calculating the Euclidean distance between the ideal location and the other location
4) The function uses the floats.Distance function from the gonum.org/v1/gonum/floats package to calculate the Euclidean distance

We also define the ranges of the features to normalize the features before calculating the similarity score

In [13]:
// Function to calculate similarity score
func calculateSimilarity(ideal Location, other Location) float64 {
	// Define the ranges of the features
	ranges := map[string]float64{
		"AverageTemperature":  40.0,
		"NrPredators":       20.0,
		"NrFoodSources":       20.0,
		"NrWaterSources":      10.0,
		"NrHumans":            50.0,
		"VegetationType":      6.0,
		"LandSize":            1000.0,
	}

	// Normalize the features
	idealVector := []float64{
		ideal.AverageTemperature / ranges["AverageTemperature"],
		ideal.NrPredators / ranges["NrPredators"],
		ideal.NrFoodSources / ranges["NrFoodSources"],
		ideal.NrWaterSources / ranges["NrWaterSources"],
		ideal.NrHumans / ranges["NrHumans"],
		ideal.VegetationType / ranges["VegetationType"],
		ideal.LandSize / ranges["LandSize"],
	}
	otherVector := []float64{
		other.AverageTemperature / ranges["AverageTemperature"],
		other.NrPredators / ranges["NrPredators"],
		other.NrFoodSources / ranges["NrFoodSources"],
		other.NrWaterSources / ranges["NrWaterSources"],
		other.NrHumans / ranges["NrHumans"],
		other.VegetationType / ranges["VegetationType"],
		other.LandSize / ranges["LandSize"],
	}

	return 1 - floats.Distance(idealVector, otherVector, 2) // Using Euclidean distance
}

The last part of the assignment is defining the function to recommend locations based on a given location, we need that to recommend locations based on a given location.

1) The function takes two parameters, the list of locations and the given location
2) Inside the function, we declare the ideal location and set it to the given location
3) We loop through the list of locations and calculate the similarity score between the ideal location and each location
4) We sort the recommendations by similarity score
5) We return the recommendations
6) We print the top 5 recommendations

In [14]:
// Function to recommend locations based on a given location
func recommendBasedOnLocation(locations []Location, givenLocation string) []struct {
	Location   Location
	Similarity float64
} {
	// Find the given location
	var ideal Location
	for _, location := range locations {
		if location.Name == givenLocation {
			ideal = location
			break
		}
	}
	if ideal.Name == "" {
		fmt.Printf("Location '%s' not found.\n", givenLocation)
		return nil
	}

	recommendations := []struct {
		Location   Location
		Similarity float64
	}{}

	for _, location := range locations {
		if location.Name != ideal.Name {
			similarity := calculateSimilarity(ideal, location)
			recommendations = append(recommendations, struct {
				Location   Location
				Similarity float64
			}{location, similarity})
		}
	}

	// Sort recommendations by similarity
	sort.Slice(recommendations, func(i, j int) bool {
		return recommendations[i].Similarity > recommendations[j].Similarity
	})

	return recommendations
}